In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, GRU, Conv1D, MaxPooling1D, Embedding, Dropout, \
    MultiHeadAttention, LayerNormalization, Add, Activation, Concatenate, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from stellargraph.layer import GraphConvolution

# 假設我們有以下歷史數據
data = {
    'date': ['2023-01-01', '2023-01-02', ...],
    'inventory': [80, 90, 75, 60, 85, 100, 95, 70, 65, 55, ...],
    'demand': [100, 95, 85, 90, 110, 80, 105, 95, 75, 120, ...],
    'price': [15, 14, 16, 18, 13, 12, 14, 17, 19, 20, ...],
    'competitor_price': [14, 15, 17, 16, 14, 13, 15, 18, 17, 19, ...],
    'product_description': ['This is a high-quality product...', 'Great for outdoor use...', ...],
    'promotions': ['Buy 1 Get 1 Free', 'None', 'Summer Sale', ...],
    'online_reviews': ['Excellent product, highly recommended!', 'Poor quality, not worth the price.', ...],
    'product_graph': [Networkx Graph Object, ...]  # 產品關聯關係圖
}

df = pd.DataFrame(data)

# 對日期進行編碼
df['date'] = pd.to_datetime(df['date'])
df['dayofweek'] = df['date'].dt.dayofweek
df['month'] = df['date'].dt.month
df['dayofyear'] = df['date'].dt.dayofyear

# 對文本特徵進行編碼
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['product_description'] + df['online_reviews'])
desc_sequences = tokenizer.texts_to_sequences(df['product_description'])
review_sequences = tokenizer.texts_to_sequences(df['online_reviews'])
max_length = max(len(s) for s in desc_sequences + review_sequences)
X_desc = pad_sequences(desc_sequences, maxlen=max_length)
X_review = pad_sequences(review_sequences, maxlen=max_length)

embeddings = Embedding(input_dim=len(tokenizer.word_index) + 1,
                       output_dim=32,
                       input_length=max_length)

# 將所有特徵縮放到 0-1 範圍
scaler = MinMaxScaler()
X_num = scaler.fit_transform(df[['inventory', 'demand', 'competitor_price', 'dayofweek', 'month', 'dayofyear']])

# 建立深度學習模型
input_desc = Input(shape=(max_length,))
embed_desc = embeddings(input_desc)
conv_desc = Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')(embed_desc)
pool_desc = MaxPooling1D(pool_size=2)(conv_desc)
flat_desc = Flatten()(pool_desc)

input_review = Input(shape=(max_length,))
embed_review = embeddings(input_review)
conv_review = Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')(embed_review)
pool_review = MaxPooling1D(pool_size=2)(conv_review)
flat_review = Flatten()(pool_review)

input_num = Input(shape=(X_num.shape[1],))
pos_encoding = PositionalEncoding(X_num.shape[1])(input_num)
lstm = LSTM(64, return_sequences=True)(pos_encoding)
gru = GRU(32)(lstm)
dropout = Dropout(0.2)(gru)

# 使用圖神經網絡捕獲產品關聯關係
input_graph = Input(shape=(None,), sparse=True)
graph_conv = GraphConvolution(64, activation='relu')([input_graph, df['product_graph']])

concat = Concatenate()([flat_desc, flat_review, dropout, graph_conv])
attention = MultiHeadAttention(num_heads=8, key_dim=64)(concat, concat)
dense1 = Dense(128, activation='relu')(attention)
dropout1 = Dropout(0.3)(dense1)
dense2 = Dense(64, activation='relu')(dropout1)
dropout2 = Dropout(0.2)(dense2)
output_price = Dense(1, name='price_output')(dropout2)
output_demand = Dense(1, name='demand_output')(dropout2)

model = Model(inputs=[input_desc, input_review, input_num, input_graph], outputs=[output_price, output_demand])

# 編譯並訓練模型
model.compile(optimizer='adam', loss=['mse', 'mse'], metrics=['mae'])
X_train = [np.asarray(X_desc), np.asarray(X_review), X_num, df['product_graph'].values]
y_train = [df['price'].values, df['demand'].values]
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# 使用GAN進行數據增強和半監督學習
gan = ...  # 建立和訓練GAN模型
X_aug, y_aug = gan.generate_samples(df, n_samples=1000)
X_train_aug = X_train + X_aug
y_train_aug = y_train + y_aug
model.fit(X_train_aug, y_train_aug, epochs=50, batch_size=32)

# 使用訓練好的模型預測新的價格和需求量
new_data = {
    'inventory': 70,
    'demand': 90,
    'competitor_price': 16,
    'dayofweek': 3,
    'month': 6,
    'dayofyear': 165,
    'product_description': 'A top-rated product for summer adventures.',
    'online_reviews': ['Great product!', 'Highly recommend.', ...]
}

new_data_num = scaler.transform([list(new_data.values())[:-2]])
new_desc_sequence = tokenizer.texts_to_sequences([new_data['product_description']])
new_review_sequences = tokenizer.texts_to_sequences(new_data['online_reviews'])
new_desc_sequence​​​​​​​​​​​​​​​​